# Getting Started with Promptolution

## Welcome to Promptolution! 

Discover a powerful tool for evolving and optimizing your LLM prompts. This notebook provides a friendly introduction to Promptolution's core functionality.

We're excited to have you try Promptolution - let's get started!

## Installation
Install Promptolution with a single command

In [22]:
# ! pip install promptolution[api]

## Imports

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import pandas as pd
from promptolution.utils import ExperimentConfig
from promptolution.helpers import run_experiment
import nest_asyncio

nest_asyncio.apply()  # Required for notebook environments

## Setting Up Your Experiment

### Prepare the data

Below, we're using a subsample of the subjectivity dataset from Hugging Face as an example. When using your own dataset, simply ensure you name the input column "x" and the target column "y", and provide a brief description of your task, that will parsed to the meta-llm during optimization.

In [25]:
df = pd.DataFrame(
    {
        "x": [str(i) for i in range(30)],
    }
)

### Creating Inital Prompts

We've defined some starter prompts below, but feel free to experiment! You might also want to explore create_prompts_from_samples to automatically generate initial prompts based on your data.

In [26]:
init_prompts = [
    "Tell the user a joke, that is novel and funny.",
    "Create a humorous joke for the user.",
    "Generate a joke that will make the user laugh.",
    "Craft a joke that is both funny and original.",
    "Invent a joke that is amusing and unique.",
    "Tell a joke to the user, that is unexpected and humorous.",
    "Devise a joke that is entertaining and fresh.",
]

### Configure Your LLM

Promptolution offers three flexible ways to access language models:

1. Local LLMs (using the Transformers library)
1. vLLM backend (for efficient serving of large language models)
1. API-based LLMs (compatible with any provider following the OpenAI standard)

For this demonstration, we'll use the DeepInfra API, but you can easily switch to other providers like Anthropic or OpenAI by simply changing the base_url and llm string in the configuration.

In [27]:
api_key = "YOUR_API_KEY"  # Replace with your Promptolution API key

with open("../deepinfratoken.txt", "r") as f:
    api_key = f.read().strip()

Here's an explanation of each configuration parameter in the ExperimentConfig:
- `optimizer`: The algorithm used for prompt optimization. Currently we support "capo", "evopromptga", "evopromptde", and "opro". For this example, we use "capo" as it is capable of leveraging few-shot examples.
- `task_description`: A string describing the task you're optimizing prompts for. This is used to provide the meta-llm with context about your task.
- `prompts`: A list of initial prompt strings that will be used as the starting point for optimization.
- `n_steps`: The number of optimization steps to run. Higher values allow more exploration and refinement but require more API calls and computational resources.
- `api_url`: The API endpoint URL used to access the language model. This example uses DeepInfra's API which follows the OpenAI standard.
- `llm`: The LLM to use for the experiment, as both downstream and meta LLM.
- `token`: Your API authentication token required to access the language model service.

In [28]:
config = ExperimentConfig(
    optimizer="evopromptga",
    task_description="Tell the user a joke!",
    prompts=init_prompts,
    n_steps=3,
    n_subsamples=3,
    subsample_strategy="random_subsample",
    api_url="https://api.deepinfra.com/v1/openai",
    model_id="meta-llama/Meta-Llama-3-8B-Instruct",
    api_key=api_key,
    task_type="judge",
)

## Run Your Experiment

With everything configured, you're ready to optimize your prompts! The `run_experiment` function will run the optimization and evaluate on a holdout set. You can expect this cell to take a few minutes to run.

In [29]:
prompts = run_experiment(df, config)

🔥 Starting optimization...
Failed to parse score from judge response: .getResponse:

<final_answer>+5</final_answer>

Reasoning:

The response addresses the task correctly by providing a joke as requested. The joke is also relevant to the input, as it is a play on words related to technology (GPS). The response is comprehensive, including the setup, punchline, and explanation of the joke, which adds to its effectiveness and makes it more enjoyable for the user. Overall, the response is a perfect match for the task, hence the score of +5.. Using 0.0 as default.
Failed to parse score from judge response: <Evaluation>

The response is a joke, and it does attempt to tell the user a joke as per the task. The joke combines two famous scientific concepts, Pavlov's dogs and Schrödinger's cat, in a humorous way. The response is creative and clever, and it does bring a smile to the face.

However, the response does not explicitly state "0" as the input, which is a requirement according to the ta

In [32]:
pd.set_option("display.max_colwidth", None)  # Show full text in DataFrame
prompts

,prompt,score
0,Generate a joke that will make the user laugh.,5.000000
1,"Conceive a fresh, humorous joke that will have a positive impact on the user's mood.",4.666667
2,Tell an unorthodox yet hilarious joke that will surprise and entertain the user.,4.666667
3,Create an unexpected yet comedic joke that will bring a smile to the user's face.,4.666667
4,Craft a witty and unexpected joke to amuse the audience.,4.166667
5,"Surprise the user with a witty, side-splitting joke that will leave them in stitches!",4.000000
6,"Tell a joke to the user, that is unexpected and humorous.",3.833333


As you can see, most optimized prompts are semantically very similar, however they often differ heavily in performance. This is exactly what we observed in our experiments across various LLMs and datasets. Running prompt optimization is an easy way to gain significant performance improvements on your task for free!

If you run into any issues while using Promptolution, please feel free to contact us. We're also happy to receive support through pull requests and other contributions to the project.


Happy prompt optimizing! 🚀✨ We can't wait to see what you build with Promptolution! 🤖💡